In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from scipy.stats import linregress
from pprint import pprint
import calendar
from calendar import month_abbr

In [2]:
Weather_Path = "Clean Data/SAWeatherAll.csv"
Drought_Path = "Clean Data/BexarCountyDrought.csv"
J17_well_Path = "Clean Data/j17.csv"


Weather_Data_df = pd.read_csv(Weather_Path, encoding="utf-8")
Drought_Data_df = pd.read_csv(Drought_Path, encoding="utf-8")
J17_Well_Data_df = pd.read_csv(J17_well_Path, encoding="utf-8")

In [3]:
#only run this ONCE you will get an error if you dont restart the kernel
#changes the months in month_abbr to lower case
lower_ma = [m.lower() for m in month_abbr]

#changes the months in df to lowercase and maps the values to numerical months
Weather_Data_df['Month'] = Weather_Data_df['Month'].str.lower().map(lambda m: lower_ma.index(m)).astype('Int8')


In [4]:
#checking to make sure the formula worked correctly, spoiler it does
#Weather_Data_df

In [5]:
#uses the apply method to merge our numerical month values into one column
cols=["Month", "Day", "Year"]
Weather_Data_df['Date'] = Weather_Data_df[cols].apply(lambda x: '/'.join(x.values.astype(str)), axis="columns")

In [6]:
#cleaning up our data frame to remove useless columns
Weather_Data_df = Weather_Data_df.drop(columns = ["Month", "Day", "Year", "Unnamed: 0"])

In [7]:
#Checking again
#Weather_Data_df

In [21]:
#convert to date time
Weather_Data_df["Date"]= pd.to_datetime(Weather_Data_df["Date"])
Weather_Data_df.dtypes

Temp Max                  int64
Temp Avg                float64
Temp Min                  int64
Humidity Max              int64
Humidity Avg            float64
Humidity Min              int64
Precipitation           float64
Date             datetime64[ns]
dtype: object

In [9]:
Weather_Data_df

,Temp Max,Temp Avg,Temp Min,Humidity Max,Humidity Avg,Humidity Min,Precipitation,Date
0,49,46.3,44,73,61.5,51,0.00,2016-01-01
1,48,45.0,43,93,67.2,48,0.00,2016-01-02
2,61,47.7,35,96,75.6,49,0.22,2016-01-03
3,57,42.7,33,93,67.6,37,0.00,2016-01-04
4,57,44.9,30,96,74.4,55,0.00,2016-01-05
...,...,...,...,...,...,...,...,...
2398,101,86.5,74,93,61.2,28,0.00,2022-07-27
2399,100,86.2,76,94,63.1,31,0.00,2022-07-28
2400,100,86.8,76,94,63.0,31,0.00,2022-07-29
2401,100,87.6,73,96,56.7,27,0.00,2022-07-30


In [18]:
#we only need to filter for last 5 years
# double check and ensure its J17 only
#clean it but removing NaNs, change DailyHighDate to datetime

J17_Well_Data_df = J17_Well_Data_df.dropna()
J17_Well_Data_df["DailyHighDate"]= pd.to_datetime(J17_Well_Data_df["DailyHighDate"])
J17_Well_Data_df.dtypes

Site                           object
DailyHighDate          datetime64[ns]
WaterLevelElevation           float64
dtype: object

In [17]:
#checking data
J17_Well_Data_df

,Site,DailyHighDate,WaterLevelElevation
0,J17WL,2022-07-05,635.98
1,J17WL,2022-07-04,636.85
2,J17WL,2022-07-03,637.15
3,J17WL,2022-07-02,636.71
4,J17WL,2022-07-01,636.55
...,...,...,...
2373,J17WL,2016-01-05,667.55
2374,J17WL,2016-01-04,667.70
2375,J17WL,2016-01-03,666.69
2376,J17WL,2016-01-02,666.37


In [22]:
#Change to datetime
Drought_Data_df[['ValidStart','ValidEnd']] = Drought_Data_df[['ValidStart','ValidEnd']].apply(pd.to_datetime)
Drought_Data_df.dtypes

map date                      int64
FIPS                          int64
County                       object
State                        object
None                        float64
D0                          float64
D1                          float64
D2                          float64
D3                          float64
D4                          float64
ValidStart           datetime64[ns]
ValidEnd             datetime64[ns]
StatisticFormatID             int64
dtype: object

In [14]:
#look at the data
Drought_Data_df

,map date,FIPS,County,State,None,D0,D1,D2,D3,D4,ValidStart,ValidEnd,StatisticFormatID
0,20220726,48029,Bexar County,TX,0.00,100.00,100.00,100.00,100.0,54.07,2022-07-26,2022-08-01,1
1,20220719,48029,Bexar County,TX,0.00,100.00,100.00,100.00,100.0,54.07,2022-07-19,2022-07-25,1
2,20220712,48029,Bexar County,TX,0.00,100.00,100.00,100.00,100.0,55.08,2022-07-12,2022-07-18,1
3,20220705,48029,Bexar County,TX,0.00,100.00,100.00,100.00,100.0,27.38,2022-07-05,2022-07-11,1
4,20220628,48029,Bexar County,TX,0.00,100.00,100.00,100.00,100.0,49.45,2022-06-28,2022-07-04,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,20170822,48029,Bexar County,TX,91.27,8.73,0.09,0.00,0.0,0.00,2017-08-22,2017-08-28,1
258,20170815,48029,Bexar County,TX,93.08,6.92,0.09,0.00,0.0,0.00,2017-08-15,2017-08-21,1
259,20170808,48029,Bexar County,TX,0.00,100.00,1.74,0.00,0.0,0.00,2017-08-08,2017-08-14,1
260,20170801,48029,Bexar County,TX,0.00,100.00,93.41,1.75,0.0,0.00,2017-08-01,2017-08-07,1


In [8]:
#Drought_Data_df

# Groupby Month and do the avg. for each month which will be utilized for daily avg. value

In [ ]:
#After we have cleaned data we will do the 10- day bucket sort to compare water days restrictons
#we are looking for the date and a "drought level" (D01, D02, D03, D04) we will use both J17_well_Data_df and Drought_Data_df

In [ ]:
#based on new data, start analyzing if the following day had higher vs lower vs equal compared to the restriction day.
#It will be a For loop thru previous cell using the +1

#Final data will be used to create plots